In [2]:
import numpy as np

import sys,os

#set current python directory, make sure you are using python 2.7 for caffe. 
caffe_root = '/local/mnt/workspace/caffe/' 
sys.path.insert(0, caffe_root + 'python')
import caffe
os.chdir(caffe_root)

caffe.set_mode_gpu()
#set the net, model and mean file
net_file=caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt'
caffe_model=caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'
mean_file=caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy'
net = caffe.Net(net_file,caffe_model,caffe.TEST)

"""
The “set_transpose” function here will transform an image from (256,256,3) to (3,256,256). 
The “set_channel_swap” function will change the channel ordering. Caffe uses BGR image format, 
so we need to change the image from RGB to BGR. If you are using OpenCV to load the image, 
then this step is not necessary since OpenCV also uses the BGR format. 
The “set_raw_scale” function normalizes the values in the image based on the 0-255 range.
"""

transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
#put channel 1 to the front
transformer.set_transpose('data', (2,0,1))
transformer.set_mean('data', np.load(mean_file).mean(1).mean(1))
#scale the image to 255
transformer.set_raw_scale('data', 255) 
#RGB->BGR
transformer.set_channel_swap('data', (2,1,0))

#load the test image and use pre-configured transformer to convert the test image as caffe input
im=caffe.io.load_image(caffe_root+'examples/images/bike.jpg')
net.blobs['data'].data[...] = transformer.preprocess('data',im)
#forward broadcasting
out = net.forward()
#get the result
out_prob = out['prob'][0]
print 'predicted calss is',out_prob.argmax()

#load the label files
imagenet_labels_filename = caffe_root + 'data/ilsvrc12/synset_words.txt'
labels = np.loadtxt(imagenet_labels_filename, str, delimiter='\t')

#get the top5 and use zip to print the top 5 probablities 
top5 = out_prob.flatten().argsort()[::-1][:5]
print "probilities and lables:"
res = zip(out_prob[top5],labels[top5])
print res

print "another way to get the top5 high probilities"
top_5 = net.blobs['prob'].data[0].flatten().argsort()[-1:-6:-1]
for i in np.arange(top_5.size):
    print (top_5[i]),"#",(labels[top_5[i]])




predicted calss is 671
probilities and lables:
[(0.97232437, 'n03792782 mountain bike, all-terrain bike, off-roader'), (0.016351981, 'n03208938 disk brake, disc brake'), (0.0075037084, 'n04482393 tricycle, trike, velocipede'), (0.0022156565, 'n02835271 bicycle-built-for-two, tandem bicycle, tandem'), (0.0013626202, 'n03785016 moped')]
another way to get the top5 high probilities
671 # n03792782 mountain bike, all-terrain bike, off-roader
535 # n03208938 disk brake, disc brake
870 # n04482393 tricycle, trike, velocipede
444 # n02835271 bicycle-built-for-two, tandem bicycle, tandem
665 # n03785016 moped
